<a href="https://colab.research.google.com/github/TheTigerHub/UTD-Summer-2025/blob/main/UTD2025_FMEA_Severity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print ("installing hugging face libraries for QLoRA")

!pip install -q accelerate==0.30.1 #hardware acceleration
print ("installed accelerate")
!pip install -q peft==0.11.1 #peft is parameter effecient fine tuning which allows LoRA and QLoRA
print ("installed peft")
!pip install -q bitsandbytes==0.43.1 #bits and bytes helps with quantization
print("installed bitsandbytes")
!pip install -q transformers==4.41.1 #hugging face library for pre-trained models, tokenizers
print("installed transformers")
!pip install -q datasets==2.19.0 #datasets is used for loading, processing, and managing datasets
print("installed datasets")
!pip install -q scikit-learn==1.4.2 # scikit-learn for data splitting
print("installed scikit-learn")
!pip install -q pandas #used for data manipulation and analysis
print("installed pandas")
!pip install -q openpyxl #read excel
print("installed openpyxl")

print ("\n libraries installed")

installing hugging face libraries for QLoRA
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 37.9 MB/s eta 0:00:00
installed accelerate
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB

In [ ]:
print ("installing hugging face libraries for QLoRA")

!pip install -q "peft[bnb]" --upgrade #installs peft pacakge with bnb extra, integrates peft and bnb

!pip install -q accelerate bitsandbytes "transformers[torch]" --upgrade #installs transformers with dependencies for PyTorch

!pip install -q datasets==2.19.0 scikit-learn==1.4.2 pandas openpyxl #installs datasets, scikit-learn, pandas, and openpyxl

print ("\n libraries installed")
print ("\n restart runtime")

installing hugging face libraries for QLoRA
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.1/362.1 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 105.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 117.4 MB/s eta 0:00:00

 libraries installed

 restart runtime


In [ ]:
#imports libraries and configures data and training

import torch #torch is necessary for the Llama model
import pandas as pd #will be used later to load, clean, and process data before converting to a hugginf face dataset
import numpy as np #library for large, multi dimensional arrays and matricies as well as operations on them
from datasets import Dataset, DatasetDict #provides effecient way to handle large datasets for machine learning, espicially natural language processing
from sklearn.model_selection import train_test_split #from sci-kit learn library, splits dataset into training and validation
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, classification_report #these provide information on the accuracy of the model
from transformers import ( #key components from transformers
    AutoTokenizer, #loads correct tokenize automatically, tokenizers convert text into numerical IDs
    AutoModelForSequenceClassification, #standard class the automatically load the correct model for sequence classification tasks
    TrainingArguments, #class to configure training
    Trainer, #class that simplifies training and eval loop for hugging face models
    DataCollatorWithPadding, #pads sequences of data in a batch to be the same legnth
    BitsAndBytesConfig # manual QLoRA config, quantization is important bc of GPU resources
)
from peft import (
    get_peft_model, #wraps model with peft adapter like QLoRA
    LoraConfig, #class for LoRA config
    TaskType, #specifies task type
    prepare_model_for_kbit_training # manual QLoRA setup with kbit quantization
)

import warnings
warnings.filterwarnings("ignore") #makes output cleaner
from huggingface_hub import notebook_login # Keep login for Llama 3.1

CSV_PATH = "2.4.xlsx" #uploaded excel file

# Column Names (Ensure these EXACTLY match your cleaned CSV/Excel headers)
COL_SUBFUNCTION = "Subfunction"
COL_REQUIREMENTS = "Requirements"
COL_FAILURE_MODE = "Potential Failure Mode and descriptions" # Base name, will be cleaned
COL_EFFECT_PRIMARY = "Potential Effect(s) of Failure (primary)" # Base name
COL_EFFECT_SECONDARY = "Potential Effect(s) of Failure (secondary)" # Base name
COL_SEVERITY = "Severity" # target column

# Input/Output Columns
INPUT_TEXT_COLS = [
    COL_SUBFUNCTION, COL_REQUIREMENTS, COL_FAILURE_MODE,
    COL_EFFECT_PRIMARY, COL_EFFECT_SECONDARY
]


#need to fill in empty cells
COLS_TO_FORWARD_FILL = [
    COL_SUBFUNCTION, COL_REQUIREMENTS, COL_FAILURE_MODE
]

#want to predict this
TARGET_COLUMN = COL_SEVERITY
NUM_LABELS = 10 # Severity can only be 1-10

# Model Config
MODEL_NAME = "meta-llama/Llama-3.2-1B" # using 3.2-1B bc 3.1 access has not been approved
MAX_SEQ_LENGTH = 512 #for memory management

# Training Config
OUTPUT_DIR = "fmea_severity_classifier_llama32_1b_standard_qlora" # <<< New output dir name
LEARNING_RATE = 1e-4      # common QLoRA starting point
NUM_EPOCHS = 3            # Train for 3 epochs which is standard for fine tuning
# MAX_STEPS = 500         # Alternative to epochs
BATCH_SIZE_PER_DEVICE = 4 # changed to 4 because model is smaller
GRAD_ACCUMULATION_STEPS = 4 # effective batch size of 16
LORA_R = 16 #the rank for LoRA matricies 16 is common
LORA_ALPHA = 32 #scaling factor for LoRA updates twice or equal to LORA_R is standard
LORA_DROPOUT = 0.05 #used for regularization, 0.05 is typical value
LOGGING_STEPS = 10 #frequent updates
SAVE_STRATEGY = "epoch" #when model should be saved
EVAL_STRATEGY = "epoch"   #when eval should be preformed
# -----done configing LoRA------


#setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #checks if device is uisng cuda GPU, cpu for backup
print(f"Using device: {device}") #prints out what device is being used
if device.type == 'cpu': print("Warning: Running on CPU!") #warning for if using cpu bc cpu training is way too slow

# Setup Label Mappings
labels_list = [str(i) for i in range(1, 11)] #creates list of strings for severity label from "1" to "10"
id2label = {i: label for i, label in enumerate(labels_list)} #maps integers 0-9 to strings "1"-"10" needed for classification head and output interpretation
label2id = {label: i for i, label in enumerate(labels_list)} #reverse of previous line, maps integers "1"-"10" to integers 0-9 which is required for training
print(f"id2label mapping: {id2label}") #print for verification
print(f"label2id mapping: {label2id}") #print for verification

# Check GPU capability for compute dtype in BNBConfig will  use float16 because t4 is the free avalible one
compute_dtype = torch.float16
if torch.cuda.is_available():
    if torch.cuda.get_device_capability()[0] >= 8: # Ampere+ (A100)
        compute_dtype = torch.bfloat16
        print("Compute dtype set to bfloat16 for Ampere+ GPU.")
    else:
        print("Compute dtype set to float16.")

Using device: cuda
id2label mapping: {0: '1', 1: '2', 2: '3', 3: '4', 4: '5', 5: '6', 6: '7', 7: '8', 8: '9', 9: '10'}
label2id mapping: {'1': 0, '2': 1, '3': 2, '4': 3, '5': 4, '6': 5, '7': 6, '8': 7, '9': 8, '10': 9}
Compute dtype set to float16.


In [ ]:
#load and Preprocess Data
#read Excel/CSV, clean header, forward filling, combine text features into a combined "text" column, and "label" column 0-9
#splits data
#converts to DatasetDict
#normalization off by default

print(f"Loading data from '{CSV_PATH}'...")


#try except block handles reading from excel and csv, and then stores it into pandas dataframe, df
try:
    try: df = pd.read_excel(CSV_PATH)
    except Exception: df = pd.read_csv(CSV_PATH)
    #this line will perform string operations to datafram column names, removes new line characters with \n, replaces multiple spaces with a single space
    #removes leading and trailing whitespace
    #saves the processed columns to cleaned_columns list
    original_columns = df.columns.tolist(); df.columns = df.columns.str.replace('\n', '', regex=False).str.replace(' +', ' ', regex=True).str.strip(); cleaned_columns = df.columns.tolist()
    #maps cleaned columns to original columns
    column_map = {clean: orig for clean, orig in zip(cleaned_columns, original_columns)}; print(f"✅ Loaded {len(df)} rows. Cleaned columns: {cleaned_columns}")
except Exception as e: print(f"❌ Error loading data: {e}"); raise #print error if reading from excel or csv produced an exception


# Function to get cleaned name robustly (optional, can hardcode if sure)
def get_cleaned_name(config_name, df_cols, original_map): # Pass original map too
    # Use split() and join() to collapse multiple spaces and remove newlines/strip
    cleaned = ' '.join(str(config_name).replace('\n', '').strip().split()) # same cleaning as before
    if cleaned not in df_cols:
         original_name = original_map.get(cleaned, config_name) # Try lookup original name if clean fails
         print(f"   Warning: Configured column '{original_config_name}' -> '{cleaned}' not found after cleaning. Check CSV/Excel headers and config variables.")
         # Fallback to original name might be safer if cleaning leads to mismatch
         if original_name in df_cols: return original_name
         return config_name # Return original config if neither found
    return cleaned

# Update configured names based on cleaned names IN THE DATAFRAME
# This has all parts of the FMEA table, and uses the funciton to get cleaned names
# ceratin cells to forward fill bc they are partialy empty in table
COL_SUBFUNCTION = get_cleaned_name(COL_SUBFUNCTION, df.columns, column_map)
COL_REQUIREMENTS = get_cleaned_name(COL_REQUIREMENTS, df.columns, column_map)
COL_FAILURE_MODE = get_cleaned_name(COL_FAILURE_MODE, df.columns, column_map)
COL_EFFECT_PRIMARY = get_cleaned_name(COL_EFFECT_PRIMARY, df.columns, column_map)
COL_EFFECT_SECONDARY = get_cleaned_name(COL_EFFECT_SECONDARY, df.columns, column_map)
COL_SEVERITY = get_cleaned_name(COL_SEVERITY, df.columns, column_map)
INPUT_TEXT_COLS = [COL_SUBFUNCTION, COL_REQUIREMENTS, COL_FAILURE_MODE, COL_EFFECT_PRIMARY, COL_EFFECT_SECONDARY]
COLS_TO_FORWARD_FILL = [COL_SUBFUNCTION, COL_REQUIREMENTS, COL_FAILURE_MODE]
TARGET_COLUMN = COL_SEVERITY # Already potentially cleaned
all_needed_columns = INPUT_TEXT_COLS + [TARGET_COLUMN]
print(f"   Using effective columns: {all_needed_columns}")

# Verify Columns Exist
missing_cols = [col for col in all_needed_columns if col not in df.columns]; #list of columns in all_needed_columns not in df.columns, which is all columns
if missing_cols: print(f"❌ Error: Columns missing: {missing_cols}"); raise ValueError("Missing columns") #print out result if missing columns is not empty, and raises error

# Preprocessing
print("⏳ Preprocessing data...")
df_selected = df[all_needed_columns].copy() #only takes needed columns
print(f"   Forward filling columns: {COLS_TO_FORWARD_FILL}...")
df_selected[COLS_TO_FORWARD_FILL] = df_selected[COLS_TO_FORWARD_FILL].ffill() #forward fills the columns that need it
initial_rows = len(df_selected); df_selected = df_selected.dropna(); final_rows = len(df_selected) #count initial rows with len(), then drops rows empty in any column, counts rows after dropping
if initial_rows > final_rows: print(f"   Dropped {initial_rows - final_rows} rows with NaN values.") #prints if rows were dropped
if final_rows == 0: raise ValueError("No data left after NaN drop") #if everything was dropped, raise an Error

# Convert Severity & Validate
try:
    df_selected[TARGET_COLUMN] = pd.to_numeric(df_selected[TARGET_COLUMN], errors='coerce') #converts values in TARGET_COLUMN into a number type, if can't turn into NaN
    df_selected = df_selected.dropna(subset=[TARGET_COLUMN]); df_selected[TARGET_COLUMN] = df_selected[TARGET_COLUMN].astype(int) #removes rows if converted to NaN
except Exception as e: print(f"❌ Error converting Severity: {e}"); raise
#filter out data not in 1-10 range and checks if any data remains
initial_rows = len(df_selected); df_selected = df_selected[df_selected[TARGET_COLUMN].between(1, 10)]; final_rows = len(df_selected)
#sets df_selected to only have rows with value 1-10, then counts the new ammount of rows
if initial_rows > final_rows: print(f"   Removed {initial_rows - final_rows} rows with Severity outside [1, 10].") #prints out how many rows were removed
if final_rows == 0: raise ValueError("No data left with valid Severity (1-10)")

# Combine Text Features
def combine_features(row):
    text_parts = [] #empty list to store the text
    for col in INPUT_TEXT_COLS: value = str(row[col]) if pd.notna(row[col]) else ""; clean_col_name = col.split('(')[0].strip(); text_parts.append(f"{clean_col_name}: {value}")
    #goes through every input column and gets the value if it is not NaN, splits at ( and removes leading and trailing whitespace then adds it to text_parts
    return "\n".join(text_parts) #returns all text parts together with new line between them as a input for the model
print("   Combining input text features into 'text' column...")
df_selected['text'] = df_selected.apply(combine_features, axis=1) #creates new column called text, fills it with the combined features in each row

# Prepare Labels (0-9)
df_selected['label'] = df_selected[TARGET_COLUMN] - 1 #converts from 1-10 to 0-9 because 0 is the first and populates new column label with those values
print(f"   Created 'label' column (0-9) from '{TARGET_COLUMN}'.")

# Keep only necessary columns
df_final = df_selected[['text', 'label']] #just needs the combined text features and the 0-9 severity value

# Create Train/Validation Split
print("⏳ Splitting data...")
train_df, valid_df = train_test_split(df_final, test_size=0.2, random_state=42, stratify=df_final['label']) #80% for training, 20% for validation severity makes sure that severity levels are evenly distributed
print(f"✅ Split complete. Train size: {len(train_df)}, Validation size: {len(valid_df)}") #prints the number of rows for training and validating

# Convert to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df, preserve_index=False)
valid_dataset = Dataset.from_pandas(valid_df, preserve_index=False)
raw_datasets = DatasetDict({'train': train_dataset, 'validation': valid_dataset}) #create DatasetDict with train and validation and corresponding datasets which is needed for Trainer
print("✅ Data prepared and converted to Hugging Face Datasets format.")
print(raw_datasets)

#  cleanup
import gc; del df, df_selected, df_final, train_df, valid_df; gc.collect() #imports garbage collector, deletes unneeded large pandas dataframes for memory saving, and runs garbage collector for memory

In [ ]:
# --- Hugging Face Login---
print("\nPlease log in to Hugging Face using an Access Token with 'read' permission.")
notebook_login()
print("✅ Login process initiated.")
# --- End Login ---
print(f"\n⏳ Loading tokenizer for '{MODEL_NAME}'...")
# Load tokenizer associated with the base model
try:
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME) # Login above handles token
    # Set padding token (Llama 3 uses EOS)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        print(f"   Tokenizer pad_token set to eos_token: {tokenizer.pad_token}")
    print("✅ Tokenizer loaded.")
except Exception as e: print(f"❌ Error loading tokenizer: {e}"); raise